<h1 align=center style="line-height:200%;font-family:vazir;color:#0099cc">
<font face="vazir" color="#0099cc">
Step1 : preprocessing
</font>
</h1>

in first step we want to do preprocessing and answer some question

In [1]:
# import libraries
import pandas as pd

In [2]:
# read the original dataset
df = pd.read_excel("sales.xlsx")

In [3]:
df

,InvoiceNumber,ProductCode,ProductName,Quantity,InvoiceDate,UnitPrice,CustomerId,Country
0,489434,85048,15CM CHRISTMAS GLASS BALL 20 LIGHTS,12,2009-12-01 07:45:00,6.95,13085.0,United Kingdom
1,489434,79323P,PINK CHERRY LIGHTS,12,2009-12-01 07:45:00,6.75,13085.0,United Kingdom
2,489434,79323W,WHITE CHERRY LIGHTS,12,2009-12-01 07:45:00,6.75,13085.0,United Kingdom
3,489434,22041,"RECORD FRAME 7"" SINGLE SIZE",48,2009-12-01 07:45:00,2.10,13085.0,United Kingdom
4,489434,21232,STRAWBERRY CERAMIC TRINKET BOX,24,2009-12-01 07:45:00,1.25,13085.0,United Kingdom
...,...,...,...,...,...,...,...,...
525456,538171,22271,FELTCRAFT DOLL ROSIE,2,2010-12-09 20:01:00,2.95,17530.0,United Kingdom
525457,538171,22750,FELTCRAFT PRINCESS LOLA DOLL,1,2010-12-09 20:01:00,3.75,17530.0,United Kingdom
525458,538171,22751,FELTCRAFT PRINCESS OLIVIA DOLL,1,2010-12-09 20:01:00,3.75,17530.0,United Kingdom
525459,538171,20970,PINK FLORAL FELTCRAFT SHOULDER BAG,2,2010-12-09 20:01:00,3.75,17530.0,United Kingdom


first we will delete rows that doesn't have <code>CustomerId</code>

In [4]:
# remove null customer id values

df_no_missing_CustomerId = df.copy()
df_no_missing_CustomerId = df_no_missing_CustomerId[df_no_missing_CustomerId["CustomerId"].notna()]
df_no_missing_CustomerId.shape

(417534, 8)

next we want remove duplicated rows

In [5]:
# keep only one row per duplicate rows
df_no_duplicate = df_no_missing_CustomerId.copy()
df_no_duplicate = df_no_duplicate.drop_duplicates()

and removing rows that has invalid prices (keep price>0)

In [6]:
# remove prices not bigger than zero
df_no_missing_price = df_no_duplicate.copy()
df_no_missing_price = df_no_missing_price[df_no_missing_price["UnitPrice"]>0]


calculate the percentage of cancelled orders.

note that because some orders have more than one product, some <code>InvoiceNumber</code> occur more than one time

In [7]:
# calculate the percentage of cancelled orders
tmp_df = df_no_missing_price.drop_duplicates(["InvoiceNumber"])
canceled_invoice=tmp_df["InvoiceNumber"].str.startswith('C').sum()
percentage_cancelled_orders = round(canceled_invoice / tmp_df.shape[0] *100)

percentage_cancelled_orders


19

find customer ID of top 5 Customers,who have the most orders

In [8]:
# change customer id to string
df_no_missing_price["CustomerId"]= df_no_missing_price["CustomerId"].astype(int)
df_no_missing_price["CustomerId"]= df_no_missing_price["CustomerId"].astype(str)
df_no_missing_price.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 410732 entries, 0 to 525460
Data columns (total 8 columns):
 #   Column         Non-Null Count   Dtype         
---  ------         --------------   -----         
 0   InvoiceNumber  410732 non-null  object        
 1   ProductCode    410732 non-null  object        
 2   ProductName    410732 non-null  object        
 3   Quantity       410732 non-null  int64         
 4   InvoiceDate    410732 non-null  datetime64[ns]
 5   UnitPrice      410732 non-null  float64       
 6   CustomerId     410732 non-null  object        
 7   Country        410732 non-null  object        
dtypes: datetime64[ns](1), float64(1), int64(1), object(5)
memory usage: 28.2+ MB


In [10]:
# calculate the top 5 cancelling customers

top_cancelling_customers = df_no_missing_price[df_no_missing_price["InvoiceNumber"].astype(str).str.startswith('C')].\
                                                groupby("CustomerId")["InvoiceNumber"].count().sort_values(ascending=False).head(5).index
top_cancelling_customers

Index(['15311', '14911', '17511', '14680', '13798'], dtype='object', name='CustomerId')

at the end we romove cancelled orders from data set

In [12]:
# remove the cancelled orders
df_no_canceled_inovice = df_no_missing_price.copy()

df_no_canceled_inovice = df_no_canceled_inovice.drop(df_no_canceled_inovice[df_no_canceled_inovice["InvoiceNumber"].astype(str).str.startswith('C')].index,axis=0)

save data set for next step

In [13]:
df_no_canceled_inovice.to_csv("preprocessed_sales.csv", index=False)